In [1]:
from langchain.chains import LLMChain, SequentialChain
import os

In [2]:
from prompts import prompt1, prompt2
from models import gemini_pro, generate_speech,text_to_speech, pipe#stable_diffusion_xl_base, 
from editing import concatenate_videos, video_create, extract_audio, add_text_to_video, combine_audio_video
from utils import remove_all_files, process_script

c:\Users\prudh\anaconda3\envs\gen_ai1\lib\site-packages\diffusers\models\transformers\transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
class shorts:
    def __init__(self) -> None:
        self.llm = gemini_pro
        self.image_generator = pipe#stable_diffusion_xl_base
        self.speech_generator = generate_speech

    def generate(self, topic):
        script = self.scriptWriter(topic)
        print(script)
        script = process_script(script)
        # print(script)
        print('Number of images to be generted: ', len(script['text_for_image_generation']))
        # print(script['text_for_speech_generation'])
        
        self.ImgGen(script['text_for_image_generation'])

        self.SpeechGen(script['text_for_speech_generation'])
        
        self.createVideo('outputs/images', 'outputs/speech', script)

        self.combineVideos('outputs/temp_videos/processed_temp_videos')
        return script
        

    def scriptWriter(self, topic):
        chain1 = LLMChain(llm=self.llm, prompt=prompt1, output_key='script')
        chain2 = LLMChain(llm=self.llm, prompt=prompt2, output_key='img_des')
        # chain3 = LLMChain(llm=self.llm, prompt=text_speech_prompt, output_key='text_speech')
        overall_chain = SequentialChain(
        chains=[chain1, chain2],#, chain3],
        input_variables=["topic"],
        output_variables=["script", "img_des"]
        )
        script = overall_chain.invoke(topic)
        print('script generated')
        return script

    def ImgGen(self, script):
        remove_all_files('outputs/images')
        # splits = script.split('\n\n')
        # texts = [split[3:] for split in splits[3].strip().split(':')[1].strip('**').strip().split('\n')]
        for i, text in enumerate(script):
            image = self.image_generator(text, num_inference_steps=30, guidance_scale=2, width=720, height=1280, verbose=0).images[0]
            image.save(f'./outputs/images/image_{i}.jpg')
        print('images generated')

    def SpeechGen(self, script):
        remove_all_files('outputs/speech')
        # splits = script.split('\n\n')
        # texts = [split[3:] for split in splits[4].strip().split(':')[1].strip('**').strip().split('\n')]
        for i, text in enumerate(script):
            self.speech_generator(text, num=i)
        print('speech generated')

    def createVideo(self, image_dir, speech_dir, script):
        remove_all_files('outputs/temp_videos')
        remove_all_files('outputs/temp_videos/processed_temp_videos')
        images = os.listdir(image_dir)
        speeches = os.listdir(speech_dir)
        if len(images) == len(speeches):
            for i in range(len(images)):
                video_create(os.path.join(image_dir, images[i]), os.path.join(speech_dir, speeches[i]), num=i)
                # print('creating temp videos')
                extract_audio(f'outputs/temp_videos/video_{i}.mp4', f'outputs/temp_videos/extracted_audio_{i}.mp3')
                # print('extracted audio')
                text = script['text_for_speech_generation'][i].upper()
                add_text_to_video(f'outputs/temp_videos/video_{i}.mp4', f'outputs/temp_videos/captions_video_{i}.mp4', text, outline_thickness=2, outline_color=(0, 0, 0), font_path='C:\\Users\\prudh\\Downloads\\montserrat\\Montserrat\\Montserrat-Bold.ttf')
                # print('captions added')
                combine_audio_video(f'outputs/temp_videos/captions_video_{i}.mp4', f'outputs/temp_videos/extracted_audio_{i}.mp3', f'outputs/temp_videos/processed_temp_videos/final_output_video_{i}.mp4')               
            print('created temp videos')
        else:
            print('len of images and speeches are not same')

    def combineVideos(self, videos_dir):
        remove_all_files('outputs/final_video')
        concatenate_videos(videos_dir)
        print("Combined video from individual videos")

In [7]:
s = shorts()
script = s.generate('a lady in india')

script generated
script processed
Number of images to be generted:  6


 `last_sigmas_type='zero'` is not supported for `lower_order_final=False`. Changing scheduler {self.config} to have `lower_order_final` set to True.


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

images generated
speech generated
No files to remove in 'outputs/temp_videos/processed_temp_videos'.
created temp videos
Combined video from individual videos


In [6]:
script

NameError: name 'script' is not defined

In [5]:
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def split_text_into_chunks(text, chunk_size):
    words = text.split()
    return [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def add_text_to_video(input_video, output_video, text, duration=1, fontsize=40, fontcolor=(255, 255, 255),
                      outline_thickness=2, outline_color=(0, 0, 0),
                      font_path='C:\\Users\\prudh\\Downloads\\montserrat\\Montserrat\\Montserrat-Bold.ttf'):
    
    # Split text into chunks
    chunks = split_text_into_chunks(text, 3)  # Adjust chunk size as needed

    # Open the input video
    cap = cv2.VideoCapture(input_video)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_duration_frames = duration * fps

    # Load font
    font = ImageFont.truetype(font_path, fontsize)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to PIL image
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(frame_pil)

        # Calculate the current chunk index based on the frame number
        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        chunk_index = current_frame // chunk_duration_frames

        # Ensure chunk_index is within the range of available chunks
        if chunk_index < len(chunks):
            # Get the current chunk of text
            chunk = chunks[chunk_index]

            # Calculate text size
            text_width, text_height = draw.textsize(chunk, font=font)

            # Calculate text position
            text_x = (width - text_width) // 2
            text_y = height - 250  # Position text at the bottom

            # Check if the text exceeds the frame width
            if text_width > width:
                # Split the text into two lines
                words = chunk.split()
                half = len(words) // 2
                line1 = ' '.join(words[:half])
                line2 = ' '.join(words[half:])

                # Calculate text size for each line
                text_size_line1 = draw.textsize(line1, font=font)
                text_size_line2 = draw.textsize(line2, font=font)

                # Calculate text position for each line
                text_x_line1 = (width - text_size_line1[0]) // 2
                text_x_line2 = (width - text_size_line2[0]) // 2
                text_y = height - 250 - text_size_line1[1]  # Adjust vertical position for two lines

                # Draw outline and text for both lines
                for dx in range(-outline_thickness, outline_thickness + 1):
                    for dy in range(-outline_thickness, outline_thickness + 1):
                        if dx != 0 or dy != 0:
                            draw.text((text_x_line1 + dx, text_y + dy), line1, font=font, fill=outline_color)
                            draw.text((text_x_line2 + dx, text_y + text_size_line1[1] + dy), line2, font=font, fill=outline_color)
                
                draw.text((text_x_line1, text_y), line1, font=font, fill=fontcolor)
                draw.text((text_x_line2, text_y + text_size_line1[1]), line2, font=font, fill=fontcolor)

            else:
                # Draw outline and text for single line
                for dx in range(-outline_thickness, outline_thickness + 1):
                    for dy in range(-outline_thickness, outline_thickness + 1):
                        if dx != 0 or dy != 0:
                            draw.text((text_x + dx, text_y + dy), chunk, font=font, fill=outline_color)
                
                draw.text((text_x, text_y), chunk, font=font, fill=fontcolor)

            # Convert back to OpenCV frame
            frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

        # Write the frame with text to the output video
        out.write(frame)

    cap.release()
    out.release()

# Example usage
input_video = 'outputs/temp_videos/video_2.mp4';
output_video = 'outputs/temp_videos/captions_video_2.mp4'
text = 'Imagidksjflsdjflkjdsflkjne if a glass of wine or beer could enhance your health!'

# Ensure the correct path to Montserrat-Bold.ttf or adjust as necessary
add_text_to_video(input_video, output_video, text)

C:\Users\prudh\AppData\Local\Temp\ipykernel_13200\2601377626.py:49: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(chunk, font=font)
C:\Users\prudh\AppData\Local\Temp\ipykernel_13200\2601377626.py:64: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size_line1 = draw.textsize(line1, font=font)
C:\Users\prudh\AppData\Local\Temp\ipykernel_13200\2601377626.py:65: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size_line2 = draw.textsize(line2, font=font)
C:\Users\prudh\AppData\Local\Temp\ipykernel_13200\2601377626.py:49: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(chunk, font=font)
